In [16]:
import yfinance as yf
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Load the S&P 500 data
sp500 = yf.download('^GSPC', start='1990-01-01', end='2023-01-01')

# Add moving averages and volatility for better results
sp500['MA50'] = sp500['Close'].rolling(window=50).mean()
sp500['MA200'] = sp500['Close'].rolling(window=200).mean()
sp500['Volatility'] = sp500['Close'].rolling(window=50).std()

# Drop NaN values (not relevant to our inputs)
sp500 = sp500.dropna()

# Create the 'Tomorrow' column for the target variable
sp500['Tomorrow'] = sp500['Close'].shift(-1)

# Remove any NaN values
sp500 = sp500.dropna()

# Create the target variable
sp500['Target'] = (sp500['Tomorrow'] > sp500['Close']).astype(int)

# Create the predictors
predictors = ['Close', 'Volume', 'Open', 'High', 'Low', 'MA50', 'MA200', 'Volatility']

# Ensure there are no NaN values in predictors
sp500 = sp500.dropna(subset=predictors)

# Standardize the data
scaler = StandardScaler()
sp500[predictors] = scaler.fit_transform(sp500[predictors])

# Save the preprocessed data and scaler
sp500.to_csv('sp500_preprocessed.csv')
import joblib
joblib.dump(scaler, 'scaler.pkl')

[*********************100%%**********************]  1 of 1 completed
C:\Users\tmilagroso\AppData\Local\Temp\ipykernel_240368\70964659.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sp500['Tomorrow'] = sp500['Close'].shift(-1)


['scaler.pkl']

In [20]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Load the preprocessed data
sp500 = pd.read_csv('sp500_preprocessed.csv', index_col=0)

# Define the model
model = RandomForestClassifier(n_estimators=100, min_samples_split=10, random_state=1)

# Train the model
model.fit(sp500[predictors], sp500['Target'])

# Evaluate the model using cross-validation
scores = cross_val_score(model, sp500[predictors], sp500['Target'], cv=5, scoring='precision')
print("Cross-validated precision scores:", scores)
print("Mean precision score:", scores.mean())

# Save the trained model
joblib.dump(model, 'model.pkl')

Cross-validated precision scores: [0.53419593 0.51811594 0.49524941 0.50949914 0.53589109]
Mean precision score: 0.5185903014424744


['model.pkl']

In [19]:
import yfinance as yf
import pandas as pd
import joblib

# Define the predictors
predictors = ['Close', 'Volume', 'Open', 'High', 'Low', 'MA50', 'MA200', 'Volatility']

# Load the most recent data up to today (Month Day, Year)
recent_data = yf.download('^GSPC', start='2023-01-02', end='2024-08-06')

# Create relevant inputs for recent data
recent_data['MA50'] = recent_data['Close'].rolling(window=50).mean()
recent_data['MA200'] = recent_data['Close'].rolling(window=200).mean()
recent_data['Volatility'] = recent_data['Close'].rolling(window=50).std()
recent_data = recent_data.dropna()

# Standardize the recent data (consistent)
scaler = joblib.load('scaler.pkl')
recent_data[predictors] = scaler.transform(recent_data[predictors])

# Use the latest available data for prediction
latest_data = recent_data.iloc[-1]

# Create a DataFrame for the latest data
latest_data_df = pd.DataFrame(latest_data).T[predictors]

# Load the trained model and make the prediction for the next day
model = joblib.load('model.pkl')
prediction = model.predict(latest_data_df)

# Print the prediction
print("Prediction for the next day (1: Up, 0: Down):", prediction[0])

# Buy or do not buy
if prediction[0] == 0:
    print("Do not buy.")
elif prediction[0] == 1:
    print("Buy.")

[*********************100%%**********************]  1 of 1 completed

Prediction for the next day (1: Up, 0: Down): 0
Do not buy.



C:\Users\tmilagroso\AppData\Local\Temp\ipykernel_240368\2947949669.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recent_data[predictors] = scaler.transform(recent_data[predictors])
